In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from datetime import datetime
from csv import DictWriter
from time import sleep
from bs4 import BeautifulSoup
from webbrowser import Chrome
import pandas as pd
import re
import random


In [2]:
#Declare browser
driver=webdriver.Chrome('chromedriver.exe')

#### ----
Source_URL='https://genvita.vn/thu-thach'

#Go to the page
driver.get(Source_URL)

try: #Click the Button to show all pages
    while True:
        number_of_loops_button=driver.find_element(By.XPATH,"/html/body/main/section[2]/div/div[3]/a").click()
        sleep(1)
except: number_of_loops_button=None
sleep(random.randint(1,2))


#Get the Title and link
elems=driver.find_elements(By.CSS_SELECTOR,".title [href] ")
Title = [elem.text for elem in elems] # list of Titles
link=[elem.get_attribute('href') for elem in elems] # List of links

C:\Users\HP\AppData\Local\Temp\ipykernel_3548\3109118352.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('chromedriver.exe')


In [3]:
# Function for append new row by name of column in csv
from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='',encoding='utf-8') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [4]:
Platform_Column="News"
Source_Name="Genvita Thử Thách"
Reaction_total=0
Share_total=0
Type_Post="Post"
Author_Page="Page Admin"

In [5]:


for i in range(0,len(link)):
    row_dict_post=[]
### Type Post
#### ----   
    Title_Post=Title[i]
    driver.get(link[i])
    html_link=BeautifulSoup(driver.page_source,"html.parser")
    
    #Section chứa time
    info_challenge=html_link.find('section',class_="challenge-instruction") 
    
    try:
        # Lấy time thô
        info_time=info_challenge.find_all('div',class_="content") 
        time_clean = re.findall('\d{1,2}\/\d{1,2}\/\d{4}',info_time[1].string)[0] #hàm lấy time clean
    except: info_challenge=None
    
    # Get total comments number
    comment_total=driver.find_elements(By.CSS_SELECTOR,"body > main > div > article > section.comment-block.js-comment-block > h3 > span > label")[0].get_attribute("innerHTML") 
    # Extract content from header
    info_new1=html_link.find('section',class_="challenge-detail-header") 
    try:
        info_new1_clean=info_new1.get_text().replace("\n\n","")
    except: info_new1=None
    # Extract the following content
    info_new2=html_link.find('section',class_="challenge-detail-content") 
    try:
        info_new2_clean=info_new2.get_text().replace("\n\n","")
    except: info_new2=None
    # Merge 2 parts from the period ## This is evolve to the extract data
    Content_Post= info_new1_clean + info_new2_clean
    
    # Summary the object to add row in file
    row_dict_post=[Platform_Column,Source_Name,Source_URL,link[i],Type_Post,Title_Post,Content_Post,time_clean,Author_Page,comment_total,Reaction_total,Share_total]
    append_list_as_row('Output_genvita.csv', row_dict_post)


In [ ]:


Platform_Column="News"
Source_Name="Genvita Thử Thách"
Author_Comment=[]
Type_Comment="Comment"
Reaction_total=0
Share_total=0
Contents_Comment=[]

In [ ]:
### ---- Comment
for a in range(0,len(link)):
        driver.get(link[a])
        Title_Comment=Title[a]
        try: #Button to show more comments 
                while True:
                        click_button_comment=driver.find_element(By.XPATH,"/html/body/main/div/article/section[6]/div[3]/div[2]/a").click()
                        sleep(1)
        except: click_button_comment=None
        html_link=BeautifulSoup(driver.page_source,"html.parser")
        # Get raw comment
        comment_text_raw=html_link.find_all( name = "div", class_ = 'media-body-top') 

        for y in range(len(comment_text_raw)-1):
                #Declare a list comment for loop
                row_dict_comment=[]
                # Get clean name
                Name_comments = comment_text_raw[y].find( name = "span", class_ = 'user-name').get_text()
                # Get clean comment
                Time_comments=comment_text_raw[y].find( name = "div", class_ = 'time').get_text()
                # Get clean content
                Contents_comment=comment_text_raw[y].find( name = "p", class_ = 'comment-content').get_text()
                  
                # Summary the object to add row in file
                row_dict_comment=[Platform_Column,Source_Name,Source_URL,link[a],Type_Comment,Title_Comment,Contents_comment,Time_comments,Name_comments,Reaction_total,Share_total]
                # Add data to file
                append_list_as_row('Output_genvita.csv', row_dict_comment)